In [1]:
import yt_dlp
import pandas as pd
import os
from pathlib import Path


# Define the CSV path and output folder
csv_path = "/Users/louiscoussement/code/VERA/data/raw/speechesURLS - Sheet1.csv"
output_folder = "/Users/louiscoussement/code/VERA/data/raw/extracted-videos"


def parse_time_to_seconds(time_str):
    """
    Convert MM:SS format to total seconds.

    Args:
        time_str: Time string in MM:SS format

    Returns:
        Total seconds as integer
    """
    try:
        parts = time_str.strip().split(':')
        if len(parts) == 2:
            minutes, seconds = int(parts[0]), int(parts[1])
            return minutes * 60 + seconds
        else:
            raise ValueError(f"Invalid time format: {time_str}")
    except Exception as e:
        raise ValueError(f"Failed to parse time '{time_str}': {e}")


def download_trimmed_video(url, start_time, end_time, video_name, output_folder):
    """
    Download a trimmed segment of a video using yt-dlp with ffmpeg.

    Args:
        url: Video URL (YouTube or other supported site)
        start_time: Start time in MM:SS format
        end_time: End time in MM:SS format
        video_name: Output filename (without extension)
        output_folder: Folder to save the video
    """
    try:
        # Parse times to seconds
        start_seconds = parse_time_to_seconds(start_time)
        end_seconds = parse_time_to_seconds(end_time)

        # Convert back to HH:MM:SS format for ffmpeg
        start_hms = f"{start_seconds // 3600:02d}:{(start_seconds % 3600) // 60:02d}:{start_seconds % 60:02d}"
        end_hms = f"{end_seconds // 3600:02d}:{(end_seconds % 3600) // 60:02d}:{end_seconds % 60:02d}"

        # Ensure video_name has .mp4 extension
        video_name = str(video_name)
        if not video_name.endswith('.mp4'):
            video_name = f"{video_name}.mp4"

        # Full output path
        output_path = os.path.join(output_folder, video_name)

        # Configure yt_dlp options with ffmpeg trimming
        ydl_opts = {
            'format': 'best[ext=mp4]+bestaudio[ext=m4a]/best',
            'outtmpl': output_path,
            'merge_output_format': 'mp4',
            'external_downloader': 'ffmpeg',
            'external_downloader_args': [
                '-ss', start_hms,
                '-to', end_hms
            ],
        }

        # Download the trimmed video
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])

        print(f"✓ Successfully downloaded: {video_name} (from {start_time} to {end_time})")

    except ValueError as ve:
        print(f"✗ Time parsing error for {video_name}: {ve}")
    except yt_dlp.utils.DownloadError as de:
        print(f"✗ Download failed for {video_name}: {de}")
    except Exception as e:
        print(f"✗ Unexpected error for {video_name}: {e}")



def main():
    """
    Main function to read CSV and process each row.
    """
    # Create output folder if it doesn't exist
    Path(output_folder).mkdir(parents=True, exist_ok=True)
    print(f"Output folder: {output_folder}")

    try:
        # Read the CSV file
        df = pd.read_csv(csv_path)

        # Validate required columns
        required_columns = ['Link', 'Start-time', 'End-time', 'Video-name']
        missing_columns = [col for col in required_columns if col not in df.columns]
        if missing_columns:
            print(f"Error: Missing required columns: {missing_columns}")
            return

        print(f"Found {len(df)} videos to process\n")

        # Iterate over each row and download trimmed video
        for index, row in df.iterrows():
            print(f"[{index + 1}/{len(df)}] Processing: {row['Video-name']}")
            download_trimmed_video(
                url=row['Link'],
                start_time=row['Start-time'],
                end_time=row['End-time'],
                video_name=row['Video-name'],
                output_folder=output_folder
            )

        print(f"\n✓ Processing complete! All videos saved to '{output_folder}' folder.")

    except FileNotFoundError:
        print(f"Error: CSV file not found at '{csv_path}'")
    except pd.errors.EmptyDataError:
        print(f"Error: CSV file is empty at '{csv_path}'")
    except Exception as e:
        print(f"Error reading CSV: {e}")

if __name__ == "__main__":
    main()

Output folder: /Users/louiscoussement/code/VERA/data/raw/extracted-videos
Found 59 videos to process

[1/59] Processing: 1
[youtube] Extracting URL: https://www.youtube.com/watch?v=mTozO0GPcCI
[youtube] mTozO0GPcCI: Downloading webpage


[youtube] mTozO0GPcCI: Downloading android sdkless player API JSON
[youtube] mTozO0GPcCI: Downloading web safari player API JSON


[youtube] mTozO0GPcCI: Downloading m3u8 information


[info] mTozO0GPcCI: Downloading 1 format(s): 96
[download] Sleeping 5.00 seconds as required by the site...
[download] Destination: /Users/louiscoussement/code/VERA/data/raw/extracted-videos/1.mp4


[hls @ 0x14d004080] Opening 'https://rr3---sn-1vo-v2vs.googlevideo.com/videoplayback/id/993a333b418f7022/itag/96/source/youtube/expire/1765219800/ei/eMk2aevOIeTi6dsPj-DIyQs/ip/195.23.151.229/requiressl/yes/ratebypass/yes/pfa/1/sgoap/clen%3D3169078%3Bdur%3D195.767%3Bgir%3Dyes%3Bitag%3D140%3Blmt%3D1739442817325793/sgovp/clen%3D41835504%3Bdur%3D195.680%3Bgir%3Dyes%3Bitag%3D137%3Blmt%3D1739442858724486/rqh/1/hls_chunk_host/rr3---sn-1vo-v2vs.googlevideo.com/xpc/EgVo2aDSNQ%3D%3D/cps/0/met/1765198200,/mh/w6/mm/31,29/mn/sn-1vo-v2vs,sn-1vo-apnz/ms/au,rdu/mv/m/mvi/3/pl/24/rms/au,au/initcwndbps/2633750/bui/AYUSA3Dd_0-dMe3eC4oFqYrOf4qMxfNZhWlX_0hnK4oU_oEFlpZPas5Dg3EAqXRvstXHP6eAM1cCRR2l/spc/wH4QqyiNBt-cfcTMYEsmc0FXhF9yRbrOC2_0pgT6vsLUVSmTxrHK9GHv/vprv/1/playlist_type/CLEAN/txp/5319224/mt/1765197893/fvip/1/keepalive/yes/fexp/51355912,51552689,51565115,51565681,51580968/sparams/expire,ei,ip,id,itag,source,requiressl,ratebypass,pfa,sgoap,sgovp,rqh,xpc,bui,spc,vprv,playlist_type/sig/AJfQdSswRgIhAPyhnD

...

[youtube] 5X-qye2AYEA: Downloading m3u8 information


[info] 5X-qye2AYEA: Downloading 1 format(s): 301
[download] Sleeping 6.00 seconds as required by the site...
[download] Destination: /Users/louiscoussement/code/VERA/data/raw/extracted-videos/59.mp4


[hls @ 0x13bf259c0] Opening 'https://rr4---sn-1vo-v2vs.googlevideo.com/videoplayback/id/e57faac9ed806040/itag/301/source/youtube/expire/1765220543/ei/X8w2aa-yAvvfvdIPh8mY0AU/ip/195.23.151.229/requiressl/yes/ratebypass/yes/pfa/1/sgoap/clen%3D2966181%3Bdur%3D183.228%3Bgir%3Dyes%3Bitag%3D140%3Blmt%3D1745439805588955/sgovp/clen%3D58767427%3Bdur%3D183.182%3Bgir%3Dyes%3Bitag%3D299%3Blmt%3D1745439814836825/rqh/1/hls_chunk_host/rr4---sn-1vo-v2vs.googlevideo.com/xpc/EgVo2aDSNQ%3D%3D/cps/105/met/1765198943,/mh/EL/mm/31,29/mn/sn-1vo-v2vs,sn-ovn-apnl/ms/au,rdu/mv/m/mvi/4/pl/24/rms/au,au/initcwndbps/1333750/bui/AYUSA3BSmUAytz6SeMmd_CmucH9jcDaDwPQE93weMSqIoCkHyCWN1VTDV9bEVw-Z-HhEmiXGyV2NNM05/spc/wH4Qq9O40Lq5WRT8VHoKr5FU6HSTgck-ScTgzmz_z8L1MlVAG9ithV0cFJNQqXc2O5c0W2YAOhX8O-Rhqq5eTAMd/vprv/1/playlist_type/CLEAN/txp/6209224/mt/1765198611/fvip/4/keepalive/yes/fexp/51355912,51552689,51565116,51565682,51580968/sparams/expire,ei,ip,id,itag,source,requiressl,ratebypass,pfa,sgoap,sgovp,rqh,xpc,bui,spc,vprv,p

[download] 100% of   23.28MiB in 00:00:09 at 2.57MiB/s
✓ Successfully downloaded: 59.mp4 (from 1:50 to 2:54)

✓ Processing complete! All videos saved to '/Users/louiscoussement/code/VERA/data/raw/extracted-videos' folder.


[out#0/mp4 @ 0x13d3c9a00] video:22724KiB audio:1019KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.393720%
frame= 3538 fps=440 q=-1.0 Lsize=   23836KiB time=00:02:54.02 bitrate=1122.1kbits/s speed=21.7x elapsed=0:00:08.03    
